In [ ]:
import pandas as pd

In [ ]:
from gimmemotifs.motif import default_motifs
motifs_db = default_motifs()
from gimmemotifs.config import DIRECT_NAME, INDIRECT_NAME
from scipy import stats

####  Figure 4 - motif correlation file

# read raw count RNA

In [ ]:
motif_basedir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC/"
all_motifs = pd.read_csv(motif_basedir + "final.out.txt",sep='\t',index_col=0)

#print(all_motifs.shape)
# exclude motifs not in footprint set 
#all_motifs = all_motifs.loc[list(set(all_motifs.index).intersection(set(heatmap_signal2.index))),] # {'GM.5.0.Unknown.0124'} is excluded
#print(all_motifs.shape)

imp_cols = ['z-score hESC',  'z-score hDE','z-score hAFE','z-score hPFE']

# all_motifs.columns = imp_cols
motifs6 = all_motifs[imp_cols]

TF_evidence_level = 'direct_and_indirect'
if TF_evidence_level == "direct_and_indirect":
        factor_kind = [DIRECT_NAME, INDIRECT_NAME]
elif TF_evidence_level == "direct":
        factor_kind = [DIRECT_NAME]

dic_motif2TFs = {}

for i in motifs_db:
        fcs = []
        for j in factor_kind:
            # print(i)
            fcs += i.factors[j]
        dic_motif2TFs[i.id] = fcs


dic_motif2TFs_clean = {}


for entry in dic_motif2TFs:
    tmp = []
    for entry2 in dic_motif2TFs[entry]:
        tmp.append(entry2.upper())
    dic_motif2TFs_clean[entry] = list(set(tmp))
    
to_del = []
for entry in dic_motif2TFs_clean.keys():
    if entry not in list(motifs6.index):
        to_del.append(entry)
        
for entry in to_del:
    del dic_motif2TFs_clean[entry]
    
list_of_tuples = []
for entry in dic_motif2TFs_clean:
    for entry2 in dic_motif2TFs_clean[entry]:
        list_of_tuples.append((entry,entry2))
        
df = pd.DataFrame(list_of_tuples, columns =['MotifName', 'TFName'])


#top_markers = []
#with open('all_top_markers_pval_0.01_logfc_1.txt','r') as f:
#    for line in f:
#        top_markers.append(line.strip())
        
rna_expr = pd.read_csv("rna_avg_tpm.csv",index_col=0)


#rna_expr = rna_expr_subset.copy()

#rna_expr['hES'] = 0.5*(rna_expr['hES_rep1'] + rna_expr['hES_rep2'])
#rna_expr['hDE'] = 0.5*(rna_expr['hDE_rep1'] + rna_expr['hDE_rep2'])
#rna_expr['hAFE'] = 0.5*(rna_expr['hAFE_rep1'] + rna_expr['hAFE_rep2'])
#rna_expr['hPFE'] = 0.5*(rna_expr['hPFE_rep1'] + rna_expr['hPFE_rep2'])

#for entry in ['hAFE_rep1', 'hES_rep2', 'hPFE_rep1', 'hDE_rep2', 'hPFE_rep2',
#       'hDE_rep1', 'hAFE_rep2', 'hES_rep1']:
#    del rna_expr[entry]




#rna_expr = rna_expr.loc[top_markers,:] # pval = 0.01, logfc=1 -> too stringent  loses Nkx2-5 etc
exclude_non_de = set(df['TFName']) - set(rna_expr.index)


df = df[~df['TFName'].isin(exclude_non_de)]
import re
df['Family'] = [re.sub("[0-9]+$","",re.sub("-|\.","",entry)) for entry in list(df['TFName'].values)]
rna_expr['Family'] = [re.sub("[0-9]+$","",re.sub("-|\.","",entry)) for entry in rna_expr.index]


df_fam = df.copy()
rna_expr_fam = pd.DataFrame(rna_expr['Family'])
rna_expr_fam['TF'] = rna_expr_fam.index
df_fam = df_fam.merge(rna_expr_fam,left_on='Family',right_on='Family')



    
del df_fam['TFName']
df_fam = df_fam.drop_duplicates()
del rna_expr['Family']
rna_expr = rna_expr.T
motifs6 = motifs6.T


df_fam['rna_motif_correlation'] = 0.0
df_fam['rna_motif_pval'] = 1.0

for entry in df_fam.index:
    
    motif_tmp = df_fam.loc[entry,'MotifName']
    tf_tmp = df_fam.loc[entry,'TF']
    
    rho, pval = stats.pearsonr(motifs6[motif_tmp].values , rna_expr[tf_tmp].values)
    
    
    
    df_fam.loc[entry,'rna_motif_correlation'] = rho
    df_fam.loc[entry,'rna_motif_pval'] = pval
    
    

# correlation with H3K27ac
motif_basedir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27ac/"
all_motifs = pd.read_csv(motif_basedir + "final.out.txt",sep='\t',index_col=0)

imp_cols = ['z-score hESC',  'z-score hDE','z-score hAFE','z-score hPFE']

# all_motifs.columns = imp_cols
motifs6 = all_motifs[imp_cols]

motifs6 = motifs6.T

df_fam['h3k27ac_motif_correlation'] = 0.0
df_fam['h3k27ac_motif_pval'] = 1.0
for entry in df_fam.index:
    
    
    
    motif_tmp = df_fam.loc[entry,'MotifName']
    tf_tmp = df_fam.loc[entry,'TF']
    
    if motif_tmp not in motifs6.columns:
        continue
    
    rho, pval = stats.pearsonr(motifs6[motif_tmp].values , rna_expr[tf_tmp].values)
    
    
    
    df_fam.loc[entry,'h3k27ac_motif_correlation'] = rho
    df_fam.loc[entry,'h3k27ac_motif_pval'] = pval
    


    
# correlation with H3K4me2
motif_basedir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K4me2/"
all_motifs = pd.read_csv(motif_basedir + "final.out.txt",sep='\t',index_col=0)

imp_cols = ['z-score hESC',  'z-score hDE','z-score hAFE','z-score hPFE']

# all_motifs.columns = imp_cols
motifs6 = all_motifs[imp_cols]

motifs6 = motifs6.T

df_fam['h3k4me2_motif_correlation'] = 0.0
df_fam['h3k4me2_motif_pval'] = 1.0
for entry in df_fam.index:
    
    
    motif_tmp = df_fam.loc[entry,'MotifName']
    tf_tmp = df_fam.loc[entry,'TF']
    
    if motif_tmp not in motifs6.columns:
        continue
        
    rho, pval = stats.pearsonr(motifs6[motif_tmp].values , rna_expr[tf_tmp].values)
    
    
    
    df_fam.loc[entry,'h3k4me2_motif_correlation'] = rho
    df_fam.loc[entry,'h3k4me2_motif_pval'] = pval
    




# correlation with H3K27me3
motif_basedir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27me3/"
all_motifs = pd.read_csv(motif_basedir + "final.out.txt",sep='\t',index_col=0)

imp_cols = ['z-score hESC',  'z-score hDE','z-score hAFE','z-score hPFE']

# all_motifs.columns = imp_cols
motifs6 = all_motifs[imp_cols]
motifs6 = motifs6.T

df_fam['h3k27me3_motif_correlation'] = 0.0
df_fam['h3k27me3_motif_pval'] = 1.0
for entry in df_fam.index:
    
    motif_tmp = df_fam.loc[entry,'MotifName']
    tf_tmp = df_fam.loc[entry,'TF']
    
    if motif_tmp not in motifs6.columns:
        continue
        
    rho, pval = stats.pearsonr(motifs6[motif_tmp].values , rna_expr[tf_tmp].values)
    
    
    
    df_fam.loc[entry,'h3k27me3_motif_correlation'] = rho
    df_fam.loc[entry,'h3k27me3_motif_pval'] = pval



# exclude non expressed genes

# exclude < 50 raw counts
# df_fam.to_csv("intermediate_no_rna_cutoffs_ryan.csv")


rna_expr_subset = pd.read_csv("/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/GRN/input/RNA_scaled_matrix_for_integration.csv",index_col=0)
 
# rna_expr = rna_expr.loc[rna_expr_subset.index,:]

raw_rna = pd.read_csv("/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/RNA//downstream_figures/raw_counts_all.csv",index_col=0)
raw_rna = raw_rna[raw_rna.max(axis=1) > 50]

# rna_expr = rna_expr.loc[list(set(raw_rna.index).intersection(rna_expr.index)),:]

print(df_fam.shape)
df_fam = df_fam[df_fam['TF'].isin(list(rna_expr_subset.index))] # in DEGs
print(df_fam.shape)
df_fam = df_fam[df_fam['TF'].isin(list(raw_rna.index))]
print(df_fam.shape)



df_fam.to_csv("all_motif_rna_correlation_fig4_FINAL.csv")

#### step 1 - correlation

In [ ]:

rna_motif_corr = 0.7

h327ac_thresh = 0.5
h3k4me2_thresh = 0.5
h3k27me3_thresh = 0.5


df2 = df_fam[(~df_fam['rna_motif_correlation'].isna()) & (df_fam['rna_motif_correlation']>rna_motif_corr) & 
            ((df_fam['h3k27ac_motif_correlation']>h327ac_thresh) | (df_fam['h3k27me3_motif_correlation']>h3k27me3_thresh) |
            (df_fam['h3k4me2_motif_correlation']>h3k4me2_thresh))]    
print(df2.shape)

#### exclude low z-score

In [ ]:
zscore_cutoff = 2

motif_basedir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC/"
all_motifs = pd.read_csv(motif_basedir + "final.out.txt",sep='\t',index_col=0)


df3 = df2.join(all_motifs,on='MotifName')
df3['motif_TF'] = df3['MotifName'].astype('str')  + '|' +  df3['TF'].astype('str')
df3 = df3.set_index('motif_TF')



df4 = df3[#(df6['% with motif']>=per_zscore) &
                   #(df6['rna_motif_correlation']>rna_corr) & 
                  #(((df6['corr hESC_rep1'] > corr_atac) & (df6['corr hESC_rep3'] > corr_atac)) |
                  #((df6['corr hDE_rep1'] > corr_atac) & (df6['corr hDE_rep4'] > corr_atac)) | 
                  #((df6['corr hAFE_rep1'] > corr_atac) & (df6['corr hAFE_rep2'] > corr_atac)) | 
                  #((df6['corr hPFE_rep1'] > corr_atac))) & 
                  ((df3['z-score hESC'] > zscore_cutoff) | # & (df6['z-score hESC_rep3'] > zscore_cutoff)) |
                  (df3['z-score hDE'] > zscore_cutoff) | # & (df6['z-score hDE_rep4'] > zscore_cutoff)) | 
                  (df3['z-score hAFE'] > zscore_cutoff) | # & (df6['z-score hAFE_rep2'] > zscore_cutoff)) | 
                  (df3['z-score hPFE'] > zscore_cutoff))]
print(df4.shape)

#### tax max motif for each TF

In [ ]:
### take max motif for each TF
idx = df4.groupby(['TF'])['rna_motif_correlation'].transform(max) == df4['rna_motif_correlation']
df5 = df4[idx]
print(df5.shape)

In [ ]:
df5.shape

In [ ]:
df5.to_csv("11222022_FINAL_with_histone/all_selected_motifs_FINAL_v2.csv") 
#filename is all_selected_motifs_FINAL_v2_150

In [ ]:
df7 = df5.join(rna_expr.T,on='TF')

df7['rna_var'] = (df7[['hES','hDE','hAFE','hPFE']].std(axis=1))   /(df7[['hES','hDE','hAFE','hPFE']].mean(axis=1))


In [ ]:
df7.to_csv("all_selected_motifs_FINAL_v2.csv")

In [ ]:
df10 = df7.sort_values('rna_var',ascending=False).groupby('MotifName').head(n=2) # .reset_index(drop=True)

print(df10.shape[0])

In [ ]:
df10.to_csv("top2_FINAL_v2_150.csv")

In [ ]:
df10 = pd.read_csv("11222022_FINAL_with_histone/top2_FINAL_v2_150.csv",index_col=0)

### plotting

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.set_figure_params(scanpy=True, dpi_save=400,dpi=150)


import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
%matplotlib inline

## plot split file

In [ ]:
df10_plot = df10[['z-score hESC','z-score hDE','z-score hAFE','z-score hPFE']]

idx_order = list(df10_plot[df10_plot.idxmax(axis=1) == 'z-score hESC'].index)
idx_order.sort()
esc_top = idx_order.copy()
print(len(esc_top))
print(esc_top)

de_max = list(df10_plot[df10_plot.idxmax(axis=1) == 'z-score hDE'].index)
tmp_idx = list(set(de_max) - set(idx_order))
tmp_idx.sort()
de_top = tmp_idx.copy()
idx_order.extend(tmp_idx)
print(len(de_top))
print(de_top)

afe_max = list(df10_plot[df10_plot.idxmax(axis=1) == 'z-score hAFE'].index)
tmp_idx = list(set(afe_max) - set(idx_order))
tmp_idx.sort()
afe_top = tmp_idx.copy()
idx_order.extend(tmp_idx)
print(len(afe_top))
print(afe_top)


pfe_max = list(df10_plot[df10_plot.idxmax(axis=1) == 'z-score hPFE'].index)
tmp_idx = list(set(pfe_max) - set(idx_order))
tmp_idx.sort()
pfe_top = tmp_idx.copy()
idx_order.extend(tmp_idx)
print(len(pfe_top))
print(pfe_top)

In [ ]:
df10_plot_master = df10_plot.copy()

# ESC plots

In [ ]:
len(esc_top)

In [ ]:
df10_plot = df10_plot_master.copy()
df10_plot = df10_plot.loc[esc_top,df10_plot.columns]

In [ ]:
df10_plot.shape

In [ ]:
df10_plot.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in df10_plot.index]
df10_plot.columns = ['hESC','hDE','hAFE','hPFE']
df10_plot.index.name = None
import anndata
adata = anndata.AnnData(X=df10_plot.T)
adata.var_names_make_unique()

In [ ]:
adata.obs['sample'] = adata.obs.index

In [ ]:
sc.settings.figdir='11222022_FINAL_with_histone/150_split_rows_new_colors/'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
figdir = '11222022_FINAL_with_histone/150_split_rows_new_colors/'

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams["savefig.dpi"] = 400
plt.figure(figsize=[4,4])

sc.pl.matrixplot(adata,var_names=adata.var.index,categories_order=['hESC','hDE','hAFE','hPFE'],groupby='sample',cmap='Purples',save='_split_atac_esc_name_fix.pdf',standard_scale='var',swap_axes=True)

In [ ]:

df10_plot = df10_plot_master.loc[esc_top,df10_plot_master.columns]
df10rna_plot = df10.loc[df10_plot.index,['hES','hDE','hAFE','hPFE']]

In [ ]:
df10rna_plot['TF'] = [entry.split('|')[1] for entry in df10rna_plot.index]

df10rna_plot = df10rna_plot.set_index('TF')
df10rna_plot.index.name = None


df10rna_plot.columns = ['hESC','hDE','hAFE','hPFE']

In [ ]:
rna_ann = df10rna_plot.T
adata_rna = anndata.AnnData(X=rna_ann)
adata_rna.var_names_make_unique()
adata_rna.obs['sample'] = adata_rna.obs.index



rna = sc.pl.matrixplot(adata_rna,var_names=adata_rna.var.index,cmap='cividis',# show_gene_labels=True,
                 categories_order=['hESC','hDE','hAFE','hPFE'],groupby='sample',save='_rna_split_esc.pdf',swap_axes=True,standard_scale='var')


In [ ]:
# rna = vars(rna)['values_df']

In [ ]:
h3k27ac_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27ac/"
h3k27ac = pd.read_csv(h3k27ac_dir + 'final.out.txt',sep='\t',index_col=0)


df10_plot = df10_plot_master.loc[esc_top,df10_plot_master.columns]

h3k27ac_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k27ac.index:
            h3k27ac_heatmap.loc[entry,'z-score hESC'] = list(h3k27ac.loc[entry2,['z-score hESC']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hDE'] = list(h3k27ac.loc[entry2,['z-score hDE']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hAFE'] = list(h3k27ac.loc[entry2,['z-score hAFE']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hPFE'] = list(h3k27ac.loc[entry2,['z-score hPFE']])[0]
        
h3k27ac_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k27ac_heatmap.index]
h3k27ac_heatmap.columns = ['hESC','hDE','hAFE','hPFE']

# h3k27ac_heatmap.loc[h3k27ac_heatmap.abs().max(axis=1) < 1,:] = 0
adata_ac = anndata.AnnData(X=h3k27ac_heatmap.T)
adata_ac.var_names_make_unique()
adata_ac.obs['sample'] = adata_ac.obs.index
h3k27ac = sc.pl.matrixplot(adata_ac,var_names=adata_ac.var.index,cmap='Greens',# show_gene_labels=True,
                 categories_order=adata_ac.obs.index,groupby='sample',swap_axes=True,save='_split_h3k27ac_esc_with_histone_cutoff.pdf',standard_scale='var')


In [ ]:
h3k4me2_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K4me2/"
h3k4me2 = pd.read_csv(h3k4me2_dir + 'final.out.txt',sep='\t',index_col=0)

df10_plot = df10_plot_master.loc[esc_top,df10_plot_master.columns]

h3k4me2_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k4me2.index:
            h3k4me2_heatmap.loc[entry,'z-score hESC'] = list(h3k4me2.loc[entry2,['z-score hESC']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hDE'] = list(h3k4me2.loc[entry2,['z-score hDE']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hAFE'] = list(h3k4me2.loc[entry2,['z-score hAFE']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hPFE'] = list(h3k4me2.loc[entry2,['z-score hPFE']])[0]

h3k4me2_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k4me2_heatmap.index]
h3k4me2_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
       
# h3k4me2_heatmap.loc[h3k4me2_heatmap.abs().max(axis=1) < 1,:] = 0
adata_h3k4me2 = anndata.AnnData(X=h3k4me2_heatmap.T)
adata_h3k4me2.var_names_make_unique()
adata_h3k4me2.obs['sample'] = adata_h3k4me2.obs.index

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np



h3k4me2 = sc.pl.matrixplot(adata_h3k4me2,var_names=adata_h3k4me2.var.index,cmap='Oranges',# show_gene_labels=True,
                 groupby='sample',categories_order=adata_h3k4me2.obs.index,swap_axes=True,save="_h3k4me2_split_esc_no_cutoff.pdf",standard_scale='var')

In [ ]:
h3k27me3_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27me3/"

h3k27me3 = pd.read_csv(h3k27me3_dir + 'final.out.txt',sep='\t',index_col=0)
h3k27me3_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

df10_plot = df10_plot_master.loc[esc_top,df10_plot_master.columns]



get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k27me3.index:
            h3k27me3_heatmap.loc[entry,'z-score hESC'] = list(h3k27me3.loc[entry2,['z-score hESC']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hDE'] = list(h3k27me3.loc[entry2,['z-score hDE']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hAFE'] = list(h3k27me3.loc[entry2,['z-score hAFE']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hPFE'] = list(h3k27me3.loc[entry2,['z-score hPFE']])[0]

h3k27me3_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k27me3_heatmap.index]
h3k27me3_heatmap.columns = ['hESC','hDE','hAFE','hPFE']


# h3k27me3_heatmap.loc[h3k27me3_heatmap.abs().max(axis=1) < 1,:] = 0

adata_h3k27me3 = anndata.AnnData(X=h3k27me3_heatmap.T)
adata_h3k27me3.var_names_make_unique()
adata_h3k27me3.obs['sample'] = adata_h3k27me3.obs.index
sc.pl.matrixplot(adata_h3k27me3,var_names=adata_h3k27me3.var.index,cmap='Reds',# show_gene_labels=True,
                 groupby='sample',categories_order=adata_h3k27me3.obs.index,swap_axes=True,save='_h3k27me3_split_esc_no_cutoff.pdf',standard_scale='var')

# DE plots

In [ ]:
df10_plot = df10_plot_master.loc[de_top,df10_plot_master.columns]

In [ ]:
df10_plot.columns = ['hESC','hDE','hAFE','hPFE']

In [ ]:
df10_plot.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in df10_plot.index]


In [ ]:
df10_plot.shape

In [ ]:
df10_plot.index.name = None
import anndata
adata = anndata.AnnData(X=df10_plot.T)
adata.var_names_make_unique()

In [ ]:
adata.obs['sample'] = adata.obs.index

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams["savefig.dpi"] = 400
plt.figure(figsize=[4,4])

sc.pl.matrixplot(adata,var_names=adata.var.index,categories_order=['hESC','hDE','hAFE','hPFE'],groupby='sample',cmap='Purples',save='_split_atac_de_name_fix.pdf',standard_scale='var',swap_axes=True)

In [ ]:
# atac = vars(motif)['values_df']

In [ ]:
df10_plot =df10_plot_master.loc[de_top,df10_plot_master.columns]
df10rna_plot = df10.loc[df10_plot.index,['hES','hDE','hAFE','hPFE']]

In [ ]:
df10rna_plot['TF'] = [entry.split('|')[1] for entry in df10rna_plot.index]
df10rna_plot = df10rna_plot.set_index('TF')
df10rna_plot.index.name = None
df10rna_plot.columns = ['hESC','hDE','hAFE','hPFE']

In [ ]:
rna_ann = df10rna_plot.T
adata_rna = anndata.AnnData(X=rna_ann)
adata_rna.var_names_make_unique()
adata_rna.obs['sample'] = adata_rna.obs.index



rna = sc.pl.matrixplot(adata_rna,var_names=adata_rna.var.index,cmap='cividis',# show_gene_labels=True,
                 categories_order=['hESC','hDE','hAFE','hPFE'],groupby='sample',save='_rna_split_de.pdf',swap_axes=True,standard_scale='var')


In [ ]:
# rna = vars(rna)['values_df']

In [ ]:
h3k27ac_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27ac/"
h3k27ac = pd.read_csv(h3k27ac_dir + 'final.out.txt',sep='\t',index_col=0)

df10_plot = df10_plot_master.loc[de_top,df10_plot_master.columns]



h3k27ac_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k27ac.index:
            h3k27ac_heatmap.loc[entry,'z-score hESC'] = list(h3k27ac.loc[entry2,['z-score hESC']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hDE'] = list(h3k27ac.loc[entry2,['z-score hDE']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hAFE'] = list(h3k27ac.loc[entry2,['z-score hAFE']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hPFE'] = list(h3k27ac.loc[entry2,['z-score hPFE']])[0]
        
h3k27ac_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k27ac_heatmap.index]
h3k27ac_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
        
# h3k27ac_heatmap.loc[h3k27ac_heatmap.abs().max(axis=1) < 1,:] = 0
adata_ac = anndata.AnnData(X=h3k27ac_heatmap.T)
adata_ac.var_names_make_unique()
adata_ac.obs['sample'] = adata_ac.obs.index
h3k27ac = sc.pl.matrixplot(adata_ac,var_names=adata_ac.var.index,cmap='Greens',# show_gene_labels=True,
                 categories_order=adata_ac.obs.index,groupby='sample',swap_axes=True,save='_split_h3k27ac_de_with_cutoff.pdf',standard_scale='var')


In [ ]:
# h3k27ac = vars(h3k27ac)['values_df']

In [ ]:
h3k4me2_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K4me2/"
h3k4me2 = pd.read_csv(h3k4me2_dir + 'final.out.txt',sep='\t',index_col=0)

df10_plot = df10_plot_master.loc[de_top,df10_plot_master.columns]

h3k4me2_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k4me2.index:
            h3k4me2_heatmap.loc[entry,'z-score hESC'] = list(h3k4me2.loc[entry2,['z-score hESC']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hDE'] = list(h3k4me2.loc[entry2,['z-score hDE']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hAFE'] = list(h3k4me2.loc[entry2,['z-score hAFE']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hPFE'] = list(h3k4me2.loc[entry2,['z-score hPFE']])[0]

h3k4me2_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k4me2_heatmap.index]
h3k4me2_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
             
# h3k4me2_heatmap.loc[h3k4me2_heatmap.abs().max(axis=1) < 1,:] = 0
adata_h3k4me2 = anndata.AnnData(X=h3k4me2_heatmap.T)
adata_h3k4me2.obs['sample'] = adata_h3k4me2.obs.index
adata_h3k4me2.var_names_make_unique()
sc.pl.matrixplot(adata_h3k4me2,var_names=adata_h3k4me2.var.index,cmap='Oranges',# show_gene_labels=True,
                 groupby='sample',categories_order=adata_h3k4me2.obs.index,swap_axes=True,save="_h3k4me2_split_de_with_histone_cut.pdf",standard_scale='var')

In [ ]:
h3k27me3_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27me3/"

h3k27me3 = pd.read_csv(h3k27me3_dir + 'final.out.txt',sep='\t',index_col=0)
h3k27me3_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

df10_plot = df10_plot_master.loc[de_top,df10_plot_master.columns]


get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k27me3.index:
            h3k27me3_heatmap.loc[entry,'z-score hESC'] = list(h3k27me3.loc[entry2,['z-score hESC']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hDE'] = list(h3k27me3.loc[entry2,['z-score hDE']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hAFE'] = list(h3k27me3.loc[entry2,['z-score hAFE']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hPFE'] = list(h3k27me3.loc[entry2,['z-score hPFE']])[0]

h3k27me3_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k27me3_heatmap.index]
h3k27me3_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
      
# h3k27me3_heatmap.loc[h3k27me3_heatmap.abs().max(axis=1) < 1,:] = 0

adata_h3k27me3 = anndata.AnnData(X=h3k27me3_heatmap.T)
adata_h3k27me3.var_names_make_unique()
adata_h3k27me3.obs['sample'] = adata_h3k27me3.obs.index
sc.pl.matrixplot(adata_h3k27me3,var_names=adata_h3k27me3.var.index,cmap='Reds',# show_gene_labels=True,
                 groupby='sample',categories_order=adata_h3k27me3.obs.index,swap_axes=True,save='_h3k27me3_split_de_with_cutoff.pdf',standard_scale='var')

# AFE

In [ ]:
df10_plot = df10_plot_master.loc[afe_top,df10_plot_master.columns]

In [ ]:
df10_plot.shape

In [ ]:
df10_plot.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in df10_plot.index]
df10_plot.columns = ['hESC','hDE','hAFE','hPFE']

In [ ]:
df10_plot.index.name = None
import anndata
adata = anndata.AnnData(X=df10_plot.T)
adata.var_names_make_unique()

In [ ]:
adata.obs['sample'] = adata.obs.index

In [ ]:
# sc.settings.figdir='11222022_FINAL/split_rows/'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
# figdir = '11222022_FINAL/split_rows/'

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams["savefig.dpi"] = 400
plt.figure(figsize=[4,4])

sc.pl.matrixplot(adata,var_names=adata.var.index,categories_order=['hESC','hDE','hAFE','hPFE'],groupby='sample',cmap='Purples',save='_split_atac_afe_name_fix.pdf',standard_scale='var',swap_axes=True)

In [ ]:
# atac = vars(motif)['values_df']

In [ ]:
df10_plot = df10_plot_master.loc[afe_top,df10_plot_master.columns]
df10rna_plot = df10.loc[df10_plot.index,['hES','hDE','hAFE','hPFE']]

In [ ]:
df10rna_plot['TF'] = [entry.split('|')[1] for entry in df10rna_plot.index]
df10rna_plot = df10rna_plot.set_index('TF')
df10rna_plot.index.name = None
df10rna_plot.columns = ['hESC','hDE','hAFE','hPFE']

In [ ]:
rna_ann = df10rna_plot.T
adata_rna = anndata.AnnData(X=rna_ann)
adata_rna.var_names_make_unique()
adata_rna.obs['sample'] = adata_rna.obs.index



rna = sc.pl.matrixplot(adata_rna,var_names=adata_rna.var.index,cmap='cividis',# show_gene_labels=True,
                 categories_order=['hESC','hDE','hAFE','hPFE'],groupby='sample',save='_rna_split_afe.pdf',swap_axes=True,standard_scale='var')


In [ ]:
# rna = vars(rna)['values_df']

In [ ]:
h3k27ac_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27ac/"
h3k27ac = pd.read_csv(h3k27ac_dir + 'final.out.txt',sep='\t',index_col=0)
h3k27ac_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

df10_plot = df10_plot_master.loc[afe_top,df10_plot_master.columns]


get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k27ac.index:
            h3k27ac_heatmap.loc[entry,'z-score hESC'] = list(h3k27ac.loc[entry2,['z-score hESC']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hDE'] = list(h3k27ac.loc[entry2,['z-score hDE']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hAFE'] = list(h3k27ac.loc[entry2,['z-score hAFE']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hPFE'] = list(h3k27ac.loc[entry2,['z-score hPFE']])[0]
        
h3k27ac_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k27ac_heatmap.index]
h3k27ac_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
       
# h3k27ac_heatmap.loc[h3k27ac_heatmap.abs().max(axis=1) < 1,:] = 0
adata_ac = anndata.AnnData(X=h3k27ac_heatmap.T)
adata_ac.var_names_make_unique()
adata_ac.obs['sample'] = adata_ac.obs.index
h3k27ac = sc.pl.matrixplot(adata_ac,var_names=adata_ac.var.index,cmap='Greens',# show_gene_labels=True,
                 categories_order=adata_ac.obs.index,groupby='sample',swap_axes=True,save='_split_h3k27ac_afe_with_cutoff.pdf',standard_scale='var')


In [ ]:
# h3k27ac = vars(h3k27ac)['values_df']

In [ ]:
h3k4me2_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K4me2/"
h3k4me2 = pd.read_csv(h3k4me2_dir + 'final.out.txt',sep='\t',index_col=0)
h3k4me2_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

df10_plot = df10_plot_master.loc[afe_top,df10_plot_master.columns]


get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k4me2.index:
            h3k4me2_heatmap.loc[entry,'z-score hESC'] = list(h3k4me2.loc[entry2,['z-score hESC']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hDE'] = list(h3k4me2.loc[entry2,['z-score hDE']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hAFE'] = list(h3k4me2.loc[entry2,['z-score hAFE']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hPFE'] = list(h3k4me2.loc[entry2,['z-score hPFE']])[0]

h3k4me2_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k4me2_heatmap.index]
h3k4me2_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
        
# h3k4me2_heatmap.loc[h3k4me2_heatmap.abs().max(axis=1) < 1,:] = 0
adata_h3k4me2 = anndata.AnnData(X=h3k4me2_heatmap.T)
adata_h3k4me2.obs['sample'] = adata_h3k4me2.obs.index
adata_h3k4me2.var_names_make_unique()
h3k4me2 = sc.pl.matrixplot(adata_h3k4me2,var_names=adata_h3k4me2.var.index,cmap='Oranges',# show_gene_labels=True,
                 groupby='sample',categories_order=adata_h3k4me2.obs.index,swap_axes=True,save="_h3k4me2_split_afe_with_cutoff.pdf",standard_scale='var')

In [ ]:
h3k27me3_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27me3/"

h3k27me3 = pd.read_csv(h3k27me3_dir + 'final.out.txt',sep='\t',index_col=0)
h3k27me3_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

df10_plot = df10_plot_master.loc[afe_top,df10_plot_master.columns]


get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k27me3.index:
            h3k27me3_heatmap.loc[entry,'z-score hESC'] = list(h3k27me3.loc[entry2,['z-score hESC']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hDE'] = list(h3k27me3.loc[entry2,['z-score hDE']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hAFE'] = list(h3k27me3.loc[entry2,['z-score hAFE']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hPFE'] = list(h3k27me3.loc[entry2,['z-score hPFE']])[0]

h3k27me3_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k27me3_heatmap.index]
h3k27me3_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
        
# h3k27me3_heatmap.loc[h3k27me3_heatmap.abs().max(axis=1) < 1,:] = 0

adata_h3k27me3 = anndata.AnnData(X=h3k27me3_heatmap.T)
adata_h3k27me3.var_names_make_unique()
adata_h3k27me3.obs['sample'] = adata_h3k27me3.obs.index
sc.pl.matrixplot(adata_h3k27me3,var_names=adata_h3k27me3.var.index,cmap='Reds',# show_gene_labels=True,
                 groupby='sample',categories_order=adata_h3k27me3.obs.index,swap_axes=True,save='_h3k27me3_split_afe_with_cutoff.pdf',standard_scale='var')

# pfe 

In [ ]:
df10_plot = df10_plot_master.loc[pfe_top,df10_plot_master.columns]

In [ ]:
df10_plot.shape

In [ ]:
df10_plot.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in df10_plot.index]
df10_plot.columns = ['hESC','hDE','hAFE','hPFE']

In [ ]:
df10_plot.index.name = None
import anndata
adata = anndata.AnnData(X=df10_plot.T)
adata.var_names_make_unique()

In [ ]:
adata.obs['sample'] = adata.obs.index

In [ ]:
# sc.settings.figdir='11222022_FINAL/split_rows/'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
# figdir = '11222022_FINAL/split_rows/'

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams["savefig.dpi"] = 400
plt.figure(figsize=[4,8])

sc.pl.matrixplot(adata,var_names=adata.var.index,categories_order=['hESC','hDE','hAFE','hPFE'],groupby='sample',cmap='Purples',save='_split_atac_pfe_name_fix.pdf',standard_scale='var',swap_axes=True)

In [ ]:
# atac = vars(motif)['values_df']

In [ ]:
df10_plot = df10_plot_master.loc[pfe_top,df10_plot_master.columns]

df10rna_plot = df10.loc[df10_plot.index,['hES','hDE','hAFE','hPFE']]

In [ ]:
df10rna_plot['TF'] = [entry.split('|')[1] for entry in df10rna_plot.index]
df10rna_plot = df10rna_plot.set_index('TF')
df10rna_plot.index.name = None
df10rna_plot.columns = ['hESC','hDE','hAFE','hPFE']

In [ ]:
rna_ann = df10rna_plot.T
adata_rna = anndata.AnnData(X=rna_ann)
adata_rna.var_names_make_unique()
adata_rna.obs['sample'] = adata_rna.obs.index



rna = sc.pl.matrixplot(adata_rna,var_names=adata_rna.var.index,cmap='cividis',# show_gene_labels=True,
                 categories_order=['hESC','hDE','hAFE','hPFE'],groupby='sample',save='_rna_split_pfe.pdf',swap_axes=True,standard_scale='var')


In [ ]:
# rna = vars(rna)['values_df']

In [ ]:
h3k27ac_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27ac/"
h3k27ac = pd.read_csv(h3k27ac_dir + 'final.out.txt',sep='\t',index_col=0)
h3k27ac_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

df10_plot = df10_plot_master.loc[pfe_top,df10_plot_master.columns]


get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k27ac.index:
            h3k27ac_heatmap.loc[entry,'z-score hESC'] = list(h3k27ac.loc[entry2,['z-score hESC']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hDE'] = list(h3k27ac.loc[entry2,['z-score hDE']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hAFE'] = list(h3k27ac.loc[entry2,['z-score hAFE']])[0]
            h3k27ac_heatmap.loc[entry,'z-score hPFE'] = list(h3k27ac.loc[entry2,['z-score hPFE']])[0]
        
h3k27ac_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k27ac_heatmap.index]
h3k27ac_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
        
     
# h3k27ac_heatmap.loc[h3k27ac_heatmap.abs().max(axis=1) < 1,:] = 0
adata_ac = anndata.AnnData(X=h3k27ac_heatmap.T)
adata_ac.var_names_make_unique()
adata_ac.obs['sample'] = adata_ac.obs.index
h3k27ac = sc.pl.matrixplot(adata_ac,var_names=adata_ac.var.index,cmap='Greens',# show_gene_labels=True,
                 categories_order=adata_ac.obs.index,groupby='sample',swap_axes=True,save='_split_h3k27ac_pfe_with_cutoff.pdf',standard_scale='var')


In [ ]:
# h3k27ac = vars(h3k27ac)['values_df']

In [ ]:
h3k4me2_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K4me2/"
h3k4me2 = pd.read_csv(h3k4me2_dir + 'final.out.txt',sep='\t',index_col=0)
h3k4me2_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

df10_plot = df10_plot_master.loc[pfe_top,df10_plot_master.columns]

get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k4me2.index:
            h3k4me2_heatmap.loc[entry,'z-score hESC'] = list(h3k4me2.loc[entry2,['z-score hESC']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hDE'] = list(h3k4me2.loc[entry2,['z-score hDE']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hAFE'] = list(h3k4me2.loc[entry2,['z-score hAFE']])[0]
            h3k4me2_heatmap.loc[entry,'z-score hPFE'] = list(h3k4me2.loc[entry2,['z-score hPFE']])[0]
        
h3k4me2_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k4me2_heatmap.index]
h3k4me2_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
        
# h3k4me2_heatmap.loc[h3k4me2_heatmap.abs().max(axis=1) < 1,:] = 0
adata_h3k4me2 = anndata.AnnData(X=h3k4me2_heatmap.T)
adata_h3k4me2.obs['sample'] = adata_h3k4me2.obs.index
adata_h3k4me2.var_names_make_unique()
h3k4me2 = sc.pl.matrixplot(adata_h3k4me2,var_names=adata_h3k4me2.var.index,cmap='Oranges',# show_gene_labels=True,
                 groupby='sample',categories_order=adata_h3k4me2.obs.index,swap_axes=True,save="_h3k4me2_split_pfe_with_cutoff.pdf",standard_scale='var')

In [ ]:
h3k27me3_dir = "/Users/LoboM/Dropbox/sharedUMass_Macrina_Rene/hESC_2021/bulk/ATAC/per_sample_integration_grn_etc/nov17_2022/input/with_filter_option_but_replicates_averaged/consensus_40k_maelstrom_op_ATAC_H3K27me3/"

h3k27me3 = pd.read_csv(h3k27me3_dir + 'final.out.txt',sep='\t',index_col=0)
h3k27me3_heatmap = pd.DataFrame(0.0,index=df10_plot.index,columns= ['z-score hESC', 'z-score hDE', 'z-score hAFE', 'z-score hPFE'])

df10_plot = df10_plot_master.loc[pfe_top,df10_plot_master.columns]


get_tf = []
for entry in df10.index:
    if entry in df10_plot.index:
        entry2 = df10.loc[entry,'MotifName']
        if entry2 in h3k27me3.index:
            h3k27me3_heatmap.loc[entry,'z-score hESC'] = list(h3k27me3.loc[entry2,['z-score hESC']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hDE'] = list(h3k27me3.loc[entry2,['z-score hDE']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hAFE'] = list(h3k27me3.loc[entry2,['z-score hAFE']])[0]
            h3k27me3_heatmap.loc[entry,'z-score hPFE'] = list(h3k27me3.loc[entry2,['z-score hPFE']])[0]

h3k27me3_heatmap.index = [entry.split('|')[0].replace('GM.5.0.','') for entry in h3k27me3_heatmap.index]
h3k27me3_heatmap.columns = ['hESC','hDE','hAFE','hPFE']
 
# h3k27me3_heatmap.loc[h3k27me3_heatmap.abs().max(axis=1) < 1,:] = 0

adata_h3k27me3 = anndata.AnnData(X=h3k27me3_heatmap.T)
adata_h3k27me3.var_names_make_unique()
adata_h3k27me3.obs['sample'] = adata_h3k27me3.obs.index
sc.pl.matrixplot(adata_h3k27me3,var_names=adata_h3k27me3.var.index,cmap='Reds',# show_gene_labels=True,
                 groupby='sample',categories_order=adata_h3k27me3.obs.index,swap_axes=True,save='_h3k27me3_split_pfe_with_cutoff.pdf',standard_scale='var')